In [1]:
from pathlib import Path
import petab_sciml
import nnUDE
import yaml

In [2]:
fp_problem_yaml = (
    Path(".").resolve().parents[3] /
    "test_cases" /
    "published" / 
    "lv_001" / 
    "problem_ude.yaml"
)
fp_problem_yaml

PosixPath('/home/maren/petab_sciml/test_cases/published/lv_001/problem_ude.yaml')

In [3]:
# import into petab_sciml

with open(fp_problem_yaml) as stream:
    problem_yaml = yaml.safe_load(stream)

fp_net_yaml = (
    fp_problem_yaml.parent / 
    problem_yaml['extensions']['petab_sciml']['net_files'][0]
)
with open(fp_net_yaml) as stream:
    net1_yaml = yaml.safe_load(stream)

nn_model = net1_yaml['models'][0]

# ensure it's a FFNN
for layer in nn_model["layers"]:
    assert layer['layer_type'] == 'Linear', \
        f"Non-linear layer: {layer}"
    
# first layer item should be a placeholder
assert nn_model["forward"][0]['op'] == 'placeholder', \
        f"First op is not 'placeholder': {nn_model['forward'][0]}"

# iterate over items in forward
# ensure at most one activation function between layers

In [4]:
net1_yaml

{'models': [{'mlmodel_id': 'model1',
   'inputs': [{'input_id': 'input1', 'transform': None}],
   'layers': [{'layer_id': 'layer1',
     'layer_type': 'Linear',
     'args': {'in_features': 2, 'out_features': 2, 'bias': True}},
    {'layer_id': 'layer2',
     'layer_type': 'Linear',
     'args': {'in_features': 2, 'out_features': 2, 'bias': True}}],
   'forward': [{'name': 'x',
     'op': 'placeholder',
     'target': 'x',
     'args': [],
     'kwargs': {}},
    {'name': 'layer1',
     'op': 'call_module',
     'target': 'layer1',
     'args': ['x'],
     'kwargs': {}},
    {'name': 'tanh',
     'op': 'call_method',
     'target': 'tanh',
     'args': ['layer1'],
     'kwargs': {}},
    {'name': 'layer2',
     'op': 'call_module',
     'target': 'layer2',
     'args': ['tanh'],
     'kwargs': {}},
    {'name': 'output',
     'op': 'output',
     'target': 'output',
     'args': ['layer2'],
     'kwargs': {}}]}]}